![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [46]:
# Import necessary packages
import pandas as pd
import numpy as np

# load data
price=pd.read_csv("data/airbnb_price.csv")
room_type=pd.ExcelFile("data/airbnb_room_type.xlsx") 
last_review=pd.read_csv("data/airbnb_last_review.tsv",sep="\t")

# parse the first sheet of the excel file into a dataframe:
df_room_type=room_type.parse(0)

# display head of each data frame
print(price.head())
print(df_room_type.head())
print(last_review.head())


   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown
   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt
   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben 

To find out the earliest and most recent views, we need to convert the `last_review` column of the `last_review` dataframe from a string to a date object, sort it in ascending order, and finally retrieve the desired dates:

In [47]:
#convert to datetime
last_review["last_review"]=pd.to_datetime(last_review["last_review"],infer_datetime_format=True,errors='coerce')

#confirm conversion
print(last_review.head())

#sort dataframe by review dates
reviews_sorted=last_review.sort_values(by="last_review").reset_index(drop=True)

#retrieve desired dates: earliest view is the first row of the new dataframe, while the latest view is the last row

early_view= reviews_sorted.loc[0,"last_review"]
recent_view=reviews_sorted.loc[len(reviews_sorted)-1,"last_review"]

   listing_id    host_name last_review
0        2595     Jennifer  2019-05-21
1        3831  LisaRoxanne  2019-07-05
2        5099        Chris  2019-06-22
3        5178     Shunichi  2019-06-24
4        5238          Ben  2019-06-09


To know how many private rooms that are listed, let us first see the vaues in `room_type`

In [48]:
print(df_room_type["room_type"].unique())

['Entire home/apt' 'private room' 'Private room' 'entire home/apt'
 'PRIVATE ROOM' 'shared room' 'ENTIRE HOME/APT' 'Shared room'
 'SHARED ROOM']


We notice that we are **private room**, **Private room**, and **PRIVATE ROOM**, and the same problem continues for the other categories regarding capitilization. To overcome this problem, we should convert all entries to lowercase letters, then get the count of number of entries of **private rooms**.

In [49]:
df_room_type["room_type"]=df_room_type["room_type"].str.lower()

#confirm the change is done
print(df_room_type["room_type"].unique())

#get number of private rooms
nb_private=(df_room_type["room_type"]=="private room").sum()

['entire home/apt' 'private room' 'shared room']


Now, Looking at the prices Dataframe

In [50]:
print(price.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown


We wish to know the average price, but the `price` column is of **string** data type. To overcome this, we first remove the string "dollars" along with the space between the numbers and "dollars". Afterwards, we convert the `price` column to `int` using `.astype()` method, and we calculate the mean price.

In [51]:
# Remove all strings from the price column
price["price"] = price["price"].str.replace(" dollars", "")

#convert the column to int
price["price"] = price["price"].astype(int)

#calculate mean, rounded to the nearest two decimal places
mean=price["price"].mean().round(2)
print(mean)

141.78


Now, we collect all our retrieved data into a dataframe

In [52]:
review_dates = pd.DataFrame({
    "first_reviewed": [early_view],
    "last_reviewed": [recent_view],
    "nb_private_rooms": [nb_private],
    "avg_price": [mean]
})
print(review_dates)

  first_reviewed last_reviewed  nb_private_rooms  avg_price
0     2019-01-01    2019-07-09             11356     141.78
